In [ ]:
!wget https://www.dropbox.com/s/ajlk3nsn051jquf/best_gn4model_ep8_ac80.h5?dl=1
!wget https://www.dropbox.com/s/ij04y6n0u1fmypf/easy_shallow_model_40030.h5?dl=1
!wget https://www.dropbox.com/s/8ov0v435tm3sxkv/modelgn3%20.h5?dl=1
!wget https://www.dropbox.com/s/kxou89giuwokjw7/best_meta_model2_retrain_gn4.h5?dl=1

# **import**

In [ ]:
import numpy as np
import random
import csv
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten,Dropout,BatchNormalization,AveragePooling2D,concatenate,Input, concatenate,LeakyReLU, GlobalAveragePooling2D
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras import initializers
import tensorflow as tf    
from tensorflow.keras.regularizers import l2
from tensorflow import keras   
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers as layers
from tensorflow.keras.layers import BatchNormalization as batchnorm
from tensorflow.keras.utils import to_categorical as npcat
from tensorflow.keras.backend import manual_variable_initialization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.svm import SVC
import xgboost as xgb
!pip install h5py

In [ ]:
print(tf.__version__)

# **prepare data**

In [ ]:
!wget https://www.dropbox.com/s/yvu6qeu7m7insl5/data.zip?dl=1
!unzip data.zip\?dl\=1

In [ ]:
!wget https://www.dropbox.com/s/cobwr5hoxef7h5m/test.zip?dl=1
!unzip test.zip?dl=1

# **image enhance**

In [ ]:
from PIL import Image
from PIL import ImageEnhance

transform_type_dict = dict(
    brightness=ImageEnhance.Brightness, contrast=ImageEnhance.Contrast,
    sharpness=ImageEnhance.Sharpness,   color=ImageEnhance.Color
)

class ColorJitter(object):
    def __init__(self, transform_dict):
        self.transforms = [(transform_type_dict[k], transform_dict[k]) for k in transform_dict]
    
    def __call__(self, img):

        img = Image.fromarray(img.astype(np.uint8))
        out = img 

        if random.random() < 0.5:
          rand_num = np.random.uniform(0, 1, len(self.transforms))
          for i, (transformer, alpha) in enumerate(self.transforms):
              r = alpha * (rand_num[i]*2.0 - 1.0) + 1   # r in [1-alpha, 1+alpha)
              out = transformer(out).enhance(r)
        out = np.array(out,dtype=np.float32)
        return out

In [ ]:
_transform_dict = {'brightness':0.1026, 'contrast':0.0935, 'sharpness':0.8386, 'color':0.1592}
_color_jitter = ColorJitter(_transform_dict)
img_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=_color_jitter
    )
test_gen = ImageDataGenerator(rescale=1./255)

# **Load Data**

## **make list**

In [ ]:
train = open('/content/data/train.csv')
val = open('/content/data/dev.csv')

reader = csv.reader(train)
labels = []
label={}
for line in reader:
  tmp = [line[0],line[1]]
  labels.append(tmp)
train.close() 
ls1=[]
ls2=[]
ls3=[]
ls4=[]
partition = {}
for i in range(1,len(labels)):
  labels[i][1] = labels[i][1].replace("A","0")
  labels[i][1] = labels[i][1].replace("B","1")
  labels[i][1] = labels[i][1].replace("C","2")
  if random.random() <= 0.2:
    if random.random() <= 0.2:ls4.append('train/'+labels[i][0])
    else:ls2.append('train/'+labels[i][0])
  else:ls1.append('train/'+labels[i][0])
  label['train/'+labels[i][0]]=labels[i][1]


reader = csv.reader(val)
labels = []
for line in reader:
  tmp = [line[0],line[1]]
  labels.append(tmp)
val.close()

for i in range(1,len(labels)):
  labels[i][1] = labels[i][1].replace("A","0")
  labels[i][1] = labels[i][1].replace("B","1")
  labels[i][1] = labels[i][1].replace("C","2")
  ls3.append('dev/'+labels[i][0])
  label['dev/'+labels[i][0]]=labels[i][1]

partition['train']=ls1
partition['dev']=ls2
partition['test']=ls3
partition['v_val']=ls4
partition['combine']=ls1+ls2+ls4
partition['vcom']=ls2+ls4

## **data loader**

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(400,400), n_channels=3,
                 n_classes=3, shuffle=False,blend=False, aug=True):
        'Initialization'
        self.blend=blend
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.aug=aug
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels),dtype=np.float32)
        y = np.empty((self.batch_size), dtype=np.float32)
        imsize=400
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = cv2.imread('/content/data/' + ID)
            res = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB),(imsize,imsize),interpolation=cv2.INTER_LINEAR)
            t=img_to_array(res)
            X[i,] = img_to_array(res/1.)
            y[i] = self.labels[ID]

        y = npcat(y, self.n_classes)
        if self.aug==True:
          a=img_gen.flow(X, y, batch_size = 32)
        else:
          a=test_gen.flow(X, y, batch_size = 32)
        X=a[0][0]
        y=a[0][1]
        if self.blend:return [X,X,X,X,X,X],y
        else:return X,y
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(400,400), n_channels=3,
                 n_classes=3, shuffle=False,blend=False, aug=True):
        'Initialization'
        self.blend=blend
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.aug=aug
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels),dtype=np.float32)
        y = np.empty((self.batch_size), dtype=np.float32)
        imsize=400
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = cv2.imread('/content/data/' + ID)
            res = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB),(imsize,imsize),interpolation=cv2.INTER_LINEAR)
            t=img_to_array(res)
            X[i,] = img_to_array(res/1.)
            y[i] = self.labels[ID]

        y = npcat(y, self.n_classes)
        if self.aug==True:
          a=img_gen.flow(X, y, batch_size = 32)
        else:
          a=test_gen.flow(X, y, batch_size = 32)
        X=a[0][0]
        y=a[0][1]
        if self.blend:return [X,X,X,X,X,X],y
        else:return X,y

## **set parameters**

In [ ]:
# Parameters
train_params = {'dim': (400,400),
                'batch_size': 32,
                'n_classes': 3,
                'n_channels': 3,
                'shuffle': True,
                'blend':False,
                'aug':True}

test_params = {'dim': (400,400),
              'batch_size': 32,
              'n_classes': 3,
              'n_channels': 3,
              'shuffle': False,
              'blend':False,
              'aug':False}

blend_params = {'dim': (400,400),
              'batch_size': 32,
              'n_classes': 3,
              'n_channels': 3,
              'shuffle': True,
              'blend':True,
              'aug':True}

vval_params = {'dim': (400,400),
              'batch_size': 32,
              'n_classes': 3,
              'n_channels': 3,
              'shuffle': False,
              'blend':True,
              'aug':False}

# Generators
training_generator = DataGenerator(partition['train'], label, **train_params)
validation_generator = DataGenerator(partition['vcom'], label, **test_params)
train_val_generator = DataGenerator(partition['combine'], label, **train_params)
testing_generator = DataGenerator(partition['test'], label, **test_params)
blending_generator = DataGenerator(partition['test'], label, **blend_params)
vval = DataGenerator(partition['v_val'], label, **vval_params)
vc = DataGenerator(partition['vcom'], label, **blend_params)

# **Convolution**

In [ ]:
def conv2d(net, filters, kernel_size, strides=(1, 1), padding='same'):
    net = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,use_bias=False)(net)
    net = BatchNormalization(axis=3)(net)
    net = LeakyReLU(alpha=0.1126)(net)
    # net = Activation('relu')(net)
    return net

#**blocks**

In [ ]:
def block_stem(inputs):
    net = conv2d(inputs, 32, (3, 3), strides=(2, 2), padding='valid')
    net = conv2d(net, 32, (3, 3), padding='valid')
    net = conv2d(net, 64, (3, 3))
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)
    branch_2 = conv2d(net, 96, (3, 3), strides=(2, 2), padding='valid')
    net = concatenate([branch_1, branch_2])
    branch_1 = conv2d(net, 64, (1, 1))
    branch_1 = conv2d(branch_1, 96, (3, 3), padding='valid')
    branch_2 = conv2d(net, 64, (1, 1))
    branch_2 = conv2d(branch_2, 64, (7, 1))
    branch_2 = conv2d(branch_2, 64, (1, 7))
    branch_2 = conv2d(branch_2, 96, (3, 3), padding='valid')
    net = concatenate([branch_1, branch_2])
    branch_1 = conv2d(net, 192, (3, 3), strides=(2, 2), padding='valid')  # different from the paper
    branch_2 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)
    net = concatenate([branch_1, branch_2])
    return net


def block_inception_a(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 96, (1, 1))

    branch_2 = conv2d(inputs, 96, (1, 1))

    branch_3 = conv2d(inputs, 64, (1, 1))
    branch_3 = conv2d(branch_3, 96, (3, 3))

    branch_4 = conv2d(inputs, 64, (1, 1))
    branch_4 = conv2d(branch_4, 96, (3, 3))
    branch_4 = conv2d(branch_4, 96, (3, 3))

    return concatenate([branch_1, branch_2, branch_3, branch_4])


def block_inception_b(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 128, (1, 1))

    branch_2 = conv2d(inputs, 384, (1, 1))

    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (1, 7))
    branch_3 = conv2d(branch_3, 256, (7, 1))  # different from the paper

    branch_4 = conv2d(inputs, 192, (1, 1))
    branch_4 = conv2d(branch_4, 192, (1, 7))
    branch_4 = conv2d(branch_4, 224, (7, 1))
    branch_4 = conv2d(branch_4, 224, (1, 7))
    branch_4 = conv2d(branch_4, 256, (7, 1))

    return concatenate([branch_1, branch_2, branch_3, branch_4])


def block_inception_c(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 256, (1, 1))

    branch_2 = conv2d(inputs, 256, (1, 1))

    branch_3 = conv2d(inputs, 384, (1, 1))
    branch_3_1 = conv2d(branch_3, 256, (1, 3))
    branch_3_2 = conv2d(branch_3, 256, (3, 1))

    branch_4 = conv2d(inputs, 384, (1, 1))
    branch_4 = conv2d(branch_4, 448, (1, 3))
    branch_4 = conv2d(branch_4, 512, (3, 1))
    branch_4_1 = conv2d(branch_4, 256, (3, 1))
    branch_4_2 = conv2d(branch_4, 256, (1, 3))

    return concatenate([branch_1, branch_2, branch_3_1, branch_3_2, branch_4_1, branch_4_2])


def block_reduction_a(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    branch_2 = conv2d(inputs, 384, (3, 3), strides=(2, 2), padding='valid')

    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (3, 3))
    branch_3 = conv2d(branch_3, 256, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([branch_1, branch_2, branch_3])


def block_reduction_b(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    branch_2 = conv2d(inputs, 192, (1, 1))
    branch_2 = conv2d(branch_2, 192, (3, 3), strides=(2, 2), padding='valid')

    branch_3 = conv2d(inputs, 256, (1, 1))
    branch_3 = conv2d(branch_3, 256, (1, 7))
    branch_3 = conv2d(branch_3, 320, (7, 1))
    branch_3 = conv2d(branch_3, 320, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([branch_1, branch_2, branch_3])

# **create_model**

In [ ]:
def create(classes_num, image_height, image_width, image_channel):
    inputs = Input((image_height, image_width, image_channel))
    net = block_stem(inputs)
    for i in range(2):
        net = block_inception_a(net)
    net = block_reduction_a(net)
    for i in range(2):
        net = block_inception_b(net)
    net = block_reduction_b(net)
    for i in range(3):
        net = block_inception_c(net)
    net = AveragePooling2D((8, 8))(net)
    net = Dropout(0.2)(net)
    net = Flatten()(net)
    # Output
    x=Dense(1000,kernel_initializer=initializers.he_uniform())(net)
    x = LeakyReLU(alpha=0.1126)(x)
    outputs = Dense(units=classes_num, activation='softmax',kernel_initializer=initializers.he_uniform())(x)
    return Model(inputs, outputs, name='Inception-v4')

# **training**

In [ ]:
imsize=400
model = create(3,imsize,imsize,3)
CB = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('shallow_modelgn4_20_best.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer=Adam(lr=1e-3),loss = 'categorical_crossentropy',metrics=['accuracy'])
# History = model.fit(img_gen.flow(x_train, y_train, batch_size = 4),steps_per_epoch = len(x_train)/4,validation_data = (x_test/255,y_test), epochs = 30 )
History = model.fit_generator(generator=training_generator,validation_data=validation_generator, epochs = 30,callbacks=[es,mc] )
model.save('shallow_modelgn4.h5')

## **eval_v4**

In [ ]:
model1=load_model('shallow_modelgn4_20_best.h5')
model1.evaluate_generator(generator=testing_generator)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_gn4model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
hist = model1.fit_generator(generator=training_generator,validation_data=validation_generator, epochs = 10,callbacks=[es,mc])

# **keras v3 pretrain**

In [ ]:
# create the base pre-trained model
input_tensor = Input(shape=(400, 400, 3))
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)# False不訓練最後的dense

x = base_model.output
x = GlobalAveragePooling2D()(x)

#最後幾層fully connected
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(1024,kernel_initializer=initializers.he_uniform())(x)
x = LeakyReLU(alpha=0.2)(x)
predictions = Dense(3,activation='softmax')(x)
model = Model(input_tensor, outputs=predictions)

#只train最後幾層，其他層不動
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(lr=1e-3),loss = 'categorical_crossentropy',metrics=['accuracy'])
#train最後幾層用的
# model.fit(img_gen.flow(x_train, y_train, batch_size = 4),steps_per_epoch = len(x_train)/4,validation_data = (x_test/255,y_test), epochs = 3 )
model.fit_generator(generator=training_generator,validation_data=validation_generator, epochs = 5 )

for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True


from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=1e-3),loss = 'categorical_crossentropy',metrics=['accuracy'])

# fine-tuning整個model
History = model.fit_generator(generator=training_generator,validation_data=validation_generator, epochs = 20 )
model.save('modelgn3.h5')

## **eval_v3**

In [ ]:
model1=load_model('modelgn3.h5')
model1.evaluate_generator(generator=testing_generator)

In [ ]:
model1.save('modelgn3.h5')

# **blending**

In [ ]:
def load_all_models():
    all_models = []
    model1=load_model('best_gn4model_ep8_ac80.h5')
    model2=load_model('modelgn3.h5')
    model3=load_model('easy_shallow_model_40030.h5')
    model4=load_model('best_meta_model_1_level.h5')
    all_models.append(model1)
    all_models.append(model2)
    all_models.append(model3)
    all_models.append(model4)
    return all_models

In [ ]:
def define_stacked_model(members):
	# update all layers in all models to not be trainable
	for i in range(len(members)):
		model = members[i]
		for layer in model.layers:
			# make not trainable
			layer.trainable = False
			# rename to avoid 'unique layer name' issue
			layer._name = 'ensemble_' + str(i+1) + '_' + str(random.random())
	# define multi-headed input
	ensemble_visible = [model.input for model in members]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in members]
	merge = concatenate(ensemble_outputs)
	x = Dense(512,kernel_initializer=initializers.he_uniform())(merge)
	x = Dropout(0.15)(x)
	x = LeakyReLU(alpha=0.11)(x)# maybe dropout
	output = Dense(3, activation='softmax',kernel_initializer=initializers.he_uniform())(x)
	model = Model(inputs=ensemble_visible, outputs=output)
	model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy'])
	return model

In [ ]:
from tensorflow.keras.utils import plot_model
model=load_model('best_meta_model_2_level.h5')
plot_model(model,  to_file='model_graph.png')

In [ ]:
from keras.utils import plot_model
members=load_all_models()
stacked_model = define_stacked_model(members)
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_meta_model_2_level.h5', monitor='accuracy', mode='max', verbose=1, save_best_only=True)
hist = stacked_model.fit_generator(generator=blending_generator, epochs = 30,callbacks=[es,mc])
stacked_model.save('learning_blending1.h5')

# **SVM data**

In [ ]:
fortest = DataGenerator(partition['test'], label, **vval_params)

In [ ]:
model=load_model('best_meta_model_2_level_retraingn4.h5')
model.evaluate_generator(generator=fortest)

# **SVM**

In [ ]:
!wget https://www.dropbox.com/s/ded8ftr922jmkwr/meta_learning_blending.h5?dl=1

In [ ]:
blend_params = {'dim': (400,400),
              'batch_size': 32,
              'n_classes': 3,
              'n_channels': 3,
              'shuffle': False,
              'blend':True,
              'aug':False}
train_params = {'dim': (400,400),
              'batch_size': 32,
              'n_classes': 3,
              'n_channels': 3,
              'shuffle': True,
              'blend':True,
              'aug':True}
blending = DataGenerator(partition['test'], label, **blend_params)
training = DataGenerator(partition['vcom'], label, **train_params)

In [ ]:
def getmodelfeature(model):
  model_feat = Model(inputs=model.input,outputs=model.get_layer('dense_6').output)
  return model_feat

In [ ]:
def get_traindata(generator,model_feat):
  data = [x for x in generator]
  feature=[]
  target=[]
  for i in range(len(data)):
    pred=model_feat.predict(data[i][0])
    feature.append(pred)
    target.append(data[i][1])

  feat_train=concatenate([feature[0],feature[1]],axis=0)
  tar_train=concatenate([target[0],target[1]],axis=0)

  for i in range(2,len(feature)):
    feat_train=concatenate([feat_train,feature[i]],axis=0)
    tar_train=concatenate([tar_train,target[i]],axis=0)
  return feat_train,tar_train

In [ ]:
def get_svmscore(feature,f_y,test,t_y):
  svm = SVC(kernel='poly')
  svm.fit(feature,np.argmax(f_y,axis=1))
  return svm.score(test,np.argmax(t_y,axis=1))

In [ ]:
def get_xgbscore(feature,f_y,test,t_y):
  xb = xgb.XGBClassifier()
  xb.fit(feature,np.argmax(f_y,axis=1))
  return xb.score(test,np.argmax(t_y,axis=1))

In [ ]:
# model=load_model('meta_learning_blending.h5')
def test_model(model):
  model_feat=getmodelfeature(model)
  feat_train,tar_train=get_traindata(training,model_feat)
  feat_test,tar_test=get_traindata(blending,model_feat)
  svm_score = get_svmscore(feat_train,tar_train,feat_test,tar_test)
  # xgb_score = get_xgbscore(feat_train,tar_train,feat_test,tar_test)
  print(svm_score)
  # print(xgb_score)

In [ ]:
 test_model(model)

In [ ]:
!wget https://www.dropbox.com/s/ajlk3nsn051jquf/best_gn4model_ep8_ac80.h5?dl=1

# **Plot**

In [ ]:
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
plt.plot(hist.history['accuracy'])
# plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(hist.history['loss'])
# plt.plot(History.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.show()